# **Montagem do Azure Data Lake Storage (ADLS) no Databricks**

### **Descrição do Processo**  
Este processo tem como objetivo montar containers do **Azure Data Lake Storage (ADLS)** no Databricks, permitindo a leitura e escrita de dados diretamente nos diretórios montados.

---

### **Origem do Processo**  
A conexão é realizada utilizando autenticação **OAuth 2.0** via **Service Principal**. Os dados são montados a partir de um **container específico** do Data Lake.

---

### **Etapas do Processamento**


1. **`CMD 2 - Configuração das Credenciais de Acesso`**  
   As credenciais são obtidas de um **Azure Key Vault** por meio do `dbutils.secrets.get()`.  
   Os seguintes valores são recuperados:  
   - `clientID` → Identificação do Service Principal  
   - `tenantID` → Identificação do Tenant (Diretório)  
   - `clientSecret` → Chave secreta do Service Principal  

2. **`CMD 3 - Configuração dos Parâmetros de Autenticação`**  
   Criamos um dicionário contendo as configurações necessárias para autenticação no ADLS, utilizando **OAuth 2.0**.

3. **`CMD 3 - Função para Montar Containers do ADLS`**  
   Definição da função `mount_adls(container_name)`, que monta um container específico do ADLS no Databricks.  
   A montagem é feita no diretório `/mnt/{storage_account_name}/{container_name}`.  

---

### **Destino do Processo**  
Os containers do ADLS são montados dentro do Databricks no diretório `/mnt/{storage_account_name}/{container_name}`, permitindo acesso aos dados via **Spark** ou **DBFS**.

---

### **Observações Importantes**
- A autenticação é baseada em um **Service Principal**, que deve ter permissões apropriadas no ADLS.  
- A montagem **não é persistente** e pode ser perdida se o cluster for reiniciado. Se necessário, reexecute o script.  
- Caso o container já esteja montado, pode ser necessário desmontá-lo com:  
  ```pytho

  dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")

  ```


In [0]:
storage_account_name = "adlsstorageworksprd"

client_id       = dbutils.secrets.get('ProjetoWarehouse', 'clientID')
tenant_id       = dbutils.secrets.get('ProjetoWarehouse', 'tenantID')
client_secret   = dbutils.secrets.get('ProjetoWarehouse', 'clientSecret')

In [0]:
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": client_id,
    "fs.azure.account.oauth2.client.secret": client_secret,
    "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
}

def mount_adls(container_name):
    dbutils.fs.mount(
        source=f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
        mount_point=f"/mnt/{storage_account_name}/{container_name}",
        extra_configs=configs
    )


In [0]:
display(dbutils.fs.mounts())